Amplitude pre-processing

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor
import pywt
from scipy.stats import median_abs_deviation


# --------------------------------------------------
# 1. LOF-based Outlier Removal
# --------------------------------------------------
def lof_filter(signal, n_neighbors=20, contamination=0.02):
    signal = signal.reshape(-1, 1)

    lof = LocalOutlierFactor(
        n_neighbors=n_neighbors,
        contamination=contamination
    )
    labels = lof.fit_predict(signal)

    filtered = signal.flatten().copy()
    median_val = np.median(filtered)

    filtered[labels == -1] = median_val
    return filtered


# --------------------------------------------------
# 2. Hampel Filter
# --------------------------------------------------
def hampel_filter(signal, window_size=5, n_sigma=3):
    filtered = signal.copy()
    half_window = window_size // 2

    for i in range(len(signal)):
        start = max(i - half_window, 0)
        end = min(i + half_window + 1, len(signal))

        window = signal[start:end]
        median = np.median(window)
        mad = median_abs_deviation(window, scale='normal')

        if mad == 0:
            continue

        if np.abs(signal[i] - median) > n_sigma * mad:
            filtered[i] = median

    return filtered


# --------------------------------------------------
# 3. DWT Denoising
# --------------------------------------------------
def dwt_denoise(signal, wavelet='db5', level=2):
    coeffs = pywt.wavedec(signal, wavelet, level=level)

    sigma = np.median(np.abs(coeffs[-1])) / 0.6745
    threshold = sigma * np.sqrt(2 * np.log(len(signal)))

    denoised_coeffs = [coeffs[0]]
    for c in coeffs[1:]:
        denoised_coeffs.append(pywt.threshold(c, threshold, mode='soft'))

    reconstructed = pywt.waverec(denoised_coeffs, wavelet)
    return reconstructed[:len(signal)]


# --------------------------------------------------
# 4. Full Amplitude Preprocessing Pipeline
# --------------------------------------------------
def preprocess_amplitude_csi(signal):
    lof_cleaned = lof_filter(signal, n_neighbors=20, contamination=0.02)
    hampel_cleaned = hampel_filter(lof_cleaned, window_size=5, n_sigma=3)
    dwt_cleaned = dwt_denoise(hampel_cleaned, wavelet='db5', level=2)
    return dwt_cleaned


# --------------------------------------------------
# 5. CSV-based Processing
# --------------------------------------------------
def process_amplitude_csv(input_csv, output_csv):
    """
    Parameters:
        input_csv (str): Path to raw CSI amplitude CSV
        output_csv (str): Path to save preprocessed CSI amplitude CSV
    """
    df = pd.read_csv(input_csv)

    amp_columns = [f"amp_{i}" for i in range(1, 31)]

    for col in amp_columns:
        if col not in df.columns:
            raise ValueError(f"Missing column: {col}")

    processed_df = pd.DataFrame()

    for col in amp_columns:
        processed_df[col] = preprocess_amplitude_csi(df[col].values)

    processed_df.to_csv(output_csv, index=False)
    print(f"Preprocessed CSI amplitude saved to: {output_csv}")


# --------------------------------------------------
# Example Usage
# --------------------------------------------------
if __name__ == "__main__":

    input_file = "CSI_Amplitude.csv"
    output_file = "CSI_Preprocessed_Amplitude.csv"

    process_amplitude_csv(input_file, output_file)



Phase pre-processing

In [ ]:
import numpy as np
import pandas as pd
import pywt
from scipy.stats import linregress


# --------------------------------------------------
# 1. Phase Unwrapping
# --------------------------------------------------
def unwrap_phase(phase):
    return np.unwrap(phase)


# --------------------------------------------------
# 2. Linear Phase Correction
# --------------------------------------------------
def linear_phase_correction(phase):
    subcarrier_idx = np.arange(len(phase))
    slope, intercept, _, _, _ = linregress(subcarrier_idx, phase)
    linear_trend = slope * subcarrier_idx + intercept
    return phase - linear_trend


# --------------------------------------------------
# 3. DWT-based Denoising
# --------------------------------------------------
def dwt_phase_denoise(phase, wavelet='db5', level=2):
    coeffs = pywt.wavedec(phase, wavelet, level=level)

    sigma = np.median(np.abs(coeffs[-1])) / 0.6745
    threshold = sigma * np.sqrt(2 * np.log(len(phase)))

    denoised_coeffs = [coeffs[0]]
    for c in coeffs[1:]:
        denoised_coeffs.append(pywt.threshold(c, threshold, mode='soft'))

    reconstructed = pywt.waverec(denoised_coeffs, wavelet)
    return reconstructed[:len(phase)]


# --------------------------------------------------
# 4. Full Phase Preprocessing Pipeline
# --------------------------------------------------
def preprocess_csi_phase(phase_signal):
    unwrapped = unwrap_phase(phase_signal)
    linear_corrected = linear_phase_correction(unwrapped)
    dwt_corrected = dwt_phase_denoise(
        linear_corrected,
        wavelet='db5',
        level=2
    )
    return dwt_corrected


# --------------------------------------------------
# 5. CSV-based Processing
# --------------------------------------------------
def process_phase_csv(input_csv, output_csv):
    """
    Parameters:
        input_csv (str): Path to raw CSI phase CSV file
        output_csv (str): Path to save preprocessed CSI phase
    """
    df = pd.read_csv(input_csv)

    phase_columns = [f"phase_{i}" for i in range(1, 31)]

    for col in phase_columns:
        if col not in df.columns:
            raise ValueError(f"Missing column: {col}")

    processed_df = pd.DataFrame()

    for col in phase_columns:
        processed_df[col] = preprocess_csi_phase(df[col].values)

    processed_df.to_csv(output_csv, index=False)
    print(f"Preprocessed CSI phase saved to: {output_csv}")


# --------------------------------------------------
# Example Usage
# --------------------------------------------------
if __name__ == "__main__":
    input_file = "CSI_raw_phase.csv"
    output_file = "CSI_preprocessed_phase.csv"

    process_phase_csv(input_file, output_file)



models/resnet1d.py

In [ ]:
import torch
import torch.nn as nn


class ResidualBlock1D(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv1d(channels, channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(channels)
        self.conv2 = nn.Conv1d(channels, channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity
        return self.relu(out)


class ResNet1D(nn.Module):
    def __init__(self, in_channels=1, base_channels=64, num_blocks=4):
        super().__init__()

        self.stem = nn.Sequential(
            nn.Conv1d(in_channels, base_channels, kernel_size=7, padding=3),
            nn.BatchNorm1d(base_channels),
            nn.ReLU(inplace=True)
        )

        self.layers = nn.Sequential(
            *[ResidualBlock1D(base_channels) for _ in range(num_blocks)]
        )

        self.global_pool = nn.AdaptiveAvgPool1d(1)

    def forward(self, x):
        # x: [B, 1, 30]
        x = self.stem(x)
        x = self.layers(x)
        x = self.global_pool(x)
        return x.squeeze(-1)  # [B, C]


models/swin1d.pymodels/swin1d.py

In [ ]:
import torch
import torch.nn as nn


class SwinBlock1D(nn.Module):
    def __init__(self, dim, num_heads):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = nn.MultiheadAttention(dim, num_heads, batch_first=True)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, dim * 4),
            nn.GELU(),
            nn.Linear(dim * 4, dim)
        )

    def forward(self, x):
        # x: [B, N, C]
        attn_out, _ = self.attn(x, x, x)
        x = self.norm1(x + attn_out)
        mlp_out = self.mlp(x)
        x = self.norm2(x + mlp_out)
        return x


class SwinTransformer1D(nn.Module):
    def __init__(self, embed_dim, depth=2, num_heads=4):
        super().__init__()
        self.blocks = nn.Sequential(
            *[SwinBlock1D(embed_dim, num_heads) for _ in range(depth)]
        )

    def forward(self, x):
        return self.blocks(x)


Loading CSVs and Forward Pass

In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from models.resnet_swin import ResNetSwinFeatureExtractor


class CSIDataset(Dataset):
    def __init__(self, amp_csv, phase_csv):
        self.amp = pd.read_csv(amp_csv).values
        self.phase = pd.read_csv(phase_csv).values

    def __len__(self):
        return len(self.amp)

    def __getitem__(self, idx):
        amp = torch.tensor(self.amp[idx], dtype=torch.float32).unsqueeze(0)
        phase = torch.tensor(self.phase[idx], dtype=torch.float32).unsqueeze(0)
        return amp, phase


if __name__ == "__main__":
    dataset = CSIDataset(
        "data/CSI_Preprocessed_Amplitude.csv",
        "data/CSI_Preprocessed_Phase.csv"
    )

    loader = DataLoader(dataset, batch_size=32, shuffle=True)

    model = ResNetSwinFeatureExtractor()
    model.eval()

    for amp, phase in loader:
        features = model(amp, phase)
        print("Extracted feature shape:", features.shape)
        break


BPNN Regression Head

In [ ]:
import torch
import torch.nn as nn


class BPNN(nn.Module):
    def __init__(self, input_dim, hidden1, hidden2, output_dim=2, dropout=0.3):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden1),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Linear(hidden1, hidden2),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Linear(hidden2, output_dim)
        )

    def forward(self, x):
        return self.net(x)


Quantum-behaved Particle Swarm Optimization (QPSO)

In [ ]:
import numpy as np


class QPSO:
    def __init__(self, fitness_fn, bounds, num_particles=10, max_iter=20, beta=0.75):
        self.fitness_fn = fitness_fn
        self.bounds = np.array(bounds)
        self.num_particles = num_particles
        self.max_iter = max_iter
        self.beta = beta

        self.dim = len(bounds)
        self.particles = np.random.uniform(
            self.bounds[:, 0],
            self.bounds[:, 1],
            (num_particles, self.dim)
        )

        self.pbest = self.particles.copy()
        self.pbest_fitness = np.full(num_particles, np.inf)

        self.gbest = None
        self.gbest_fitness = np.inf

    def optimize(self):
        for t in range(self.max_iter):
            mbest = np.mean(self.pbest, axis=0)

            for i in range(self.num_particles):
                fitness = self.fitness_fn(self.particles[i])

                if fitness < self.pbest_fitness[i]:
                    self.pbest_fitness[i] = fitness
                    self.pbest[i] = self.particles[i]

                if fitness < self.gbest_fitness:
                    self.gbest_fitness = fitness
                    self.gbest = self.particles[i]

            for i in range(self.num_particles):
                u = np.random.rand(self.dim)
                phi = np.random.rand(self.dim)

                p = phi * self.pbest[i] + (1 - phi) * self.gbest
                self.particles[i] = p + self.beta * np.abs(mbest - self.particles[i]) * np.log(1 / u)

                # Bound handling
                self.particles[i] = np.clip(
                    self.particles[i],
                    self.bounds[:, 0],
                    self.bounds[:, 1]
                )

            print(f"Iter {t+1}/{self.max_iter} | Best MAE: {self.gbest_fitness:.4f}")

        return self.gbest, self.gbest_fitness


QPSO-Optimized BPNN Training

In [ ]:
import torch
import torch.nn as nn
import numpy as np

from models.bpnn import BPNN
from optimization.qpso import QPSO


# --------------------------------------------------
# Feature Loader (replace with Swin output)
import torch
import numpy as np


def load_features(
    feature_path="features/swin_features.npy",
    label_path="features/labels.npy"
):
    """
    Load Swin Transformer extracted features and ground-truth locations.

    Returns:
        X (torch.Tensor): Swin feature embeddings [N, 128]
        y (torch.Tensor): Ground-truth coordinates [N, 2]
    """
    X = np.load(feature_path)
    y = np.load(label_path)

    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)

    return X, y



# --------------------------------------------------
# Fitness Function for QPSO (Train + Validation only)
# --------------------------------------------------
def bpnn_fitness(params):
    hidden1 = int(params[0])
    hidden2 = int(params[1])
    lr = params[2]

    model = BPNN(
        input_dim=128,
        hidden1=hidden1,
        hidden2=hidden2
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.L1Loss()

    # ---- Training ----
    model.train()
    for _ in range(100):  # short training for fitness evaluation
        optimizer.zero_grad()
        pred = model(X_train)
        loss = criterion(pred, y_train)
        loss.backward()
        optimizer.step()

    # ---- Validation ----
    model.eval()
    with torch.no_grad():
        val_pred = model(X_val)
        val_loss = criterion(val_pred, y_val)

    return val_loss.item()


# --------------------------------------------------
# Main
# --------------------------------------------------
if __name__ == "__main__":

    X, y = load_features()
    N = X.shape[0]

    # --------------------------------------------------
    # 7:1:2 Split (Train : Val : Test)
    # --------------------------------------------------
    train_end = int(0.7 * N)
    val_end = int(0.8 * N)

    X_train, y_train = X[:train_end], y[:train_end]
    X_val, y_val = X[train_end:val_end], y[train_end:val_end]
    X_test, y_test = X[val_end:], y[val_end:]

    print(f"Train samples: {len(X_train)}")
    print(f"Validation samples: {len(X_val)}")
    print(f"Test samples: {len(X_test)}")

    # --------------------------------------------------
    # QPSO Hyperparameter Bounds
    # --------------------------------------------------
    bounds = [
        (32, 256),     # hidden layer 1 neurons
        (16, 128),     # hidden layer 2 neurons
        (1e-4, 1e-2)   # learning rate
    ]

    qpso = QPSO(
        fitness_fn=bpnn_fitness,
        bounds=bounds,
        num_particles=10,
        max_iter=50
    )

    best_params, best_fitness = qpso.optimize()

    print("\nOptimal BPNN Parameters (QPSO):")
    print(f"Hidden Layer 1: {int(best_params[0])}")
    print(f"Hidden Layer 2: {int(best_params[1])}")
    print(f"Learning Rate: {best_params[2]:.6f}")
    print(f"Best Validation MAE: {best_fitness:.4f}")

    # --------------------------------------------------
    # Final Training with Optimal Parameters
    # --------------------------------------------------
    final_model = BPNN(
        input_dim=128,
        hidden1=int(best_params[0]),
        hidden2=int(best_params[1])
    )

    optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params[2])
    criterion = nn.L1Loss()

    final_model.train()
    for epoch in range(200):
        optimizer.zero_grad()
        pred = final_model(X_train)
        loss = criterion(pred, y_train)
        loss.backward()
        optimizer.step()

    # --------------------------------------------------
    # Testing (Unseen Data)
    # --------------------------------------------------
    final_model.eval()
    with torch.no_grad():
        test_pred = final_model(X_test)
        test_mae = criterion(test_pred, y_test)
        test_rmse = torch.sqrt(nn.MSELoss()(test_pred, y_test))

    print("\nTest Performance:")
    print(f"Test MAE  : {test_mae.item():.4f} m")
    print(f"Test RMSE : {test_rmse.item():.4f} m")
